In [1]:
import SimpleITK as sitk
import os
#print(sitk.__version__)
import numpy as np

In [2]:
def normalize(image, mask):
    """
    perform data normalization
    :param image: input nd array
    :param mask: corresponding foreground mask
    :return: normalized array
    """
    ret = image.copy()
    image_masked = np.ma.masked_array(ret, ~(mask))
    ret[mask] = ret[mask] - np.mean(image_masked)
    ret[mask] = ret[mask] / np.var(image_masked) ** 0.5

    ret[~mask] = -100.

    return ret

In [3]:
path = '/home/localadmin/segthor/data/train_resampled2_2_25'
test_image = 'Patient_01'

fixed_image = sitk.ReadImage(os.path.join(path,test_image,test_image+'.nii.gz'))
#fixed_image_np = sitk.GetArrayFromImage(fixed_image)
#print(image_np.shape)
#fixed_image_np = normalize(fixed_image_np, fixed_image_np > -300)
#fixed_image = sitk.GetImageFromArray(fixed_image_np)

In [4]:
SimpleElastix = sitk.ElastixImageFilter()
Transformix = sitk.TransformixImageFilter()
SimpleElastix.SetFixedImage(fixed_image)


<SimpleITK.SimpleITK.ElastixImageFilter; proxy of <Swig Object of type 'itk::simple::ElastixImageFilter::Self *' at 0x7fe7c0970f00> >

In [5]:
image_names = ['Patient_0{}'.format(i) for i in range(2,9)]

In [6]:
series = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
series.sort()
print(series)

['Patient_01', 'Patient_02', 'Patient_03', 'Patient_04', 'Patient_05', 'Patient_06', 'Patient_07', 'Patient_08', 'Patient_09', 'Patient_10', 'Patient_11', 'Patient_12', 'Patient_13', 'Patient_14', 'Patient_15', 'Patient_16', 'Patient_17', 'Patient_18', 'Patient_19', 'Patient_20', 'Patient_21', 'Patient_22', 'Patient_23', 'Patient_24', 'Patient_25', 'Patient_26', 'Patient_27', 'Patient_28', 'Patient_29', 'Patient_30', 'Patient_31', 'Patient_32', 'Patient_33', 'Patient_34', 'Patient_35', 'Patient_36', 'Patient_37', 'Patient_38', 'Patient_39', 'Patient_40']


In [7]:
resultLabels = sitk.VectorOfImage()

for image_name in series[1:]:
    image = sitk.ReadImage(os.path.join(path,image_name,image_name+'.nii.gz'))
    #image_np = sitk.GetArrayFromImage(image)
    #print(image_np.shape)
    #image_np = normalize(image_np, image_np > -300)
    #image = sitk.GetImageFromArray(image_np)
    
    image_test = sitk.GetArrayFromImage(image)
    #print(np.max(image_test))
    label = sitk.ReadImage(os.path.join(path,image_name,'GT.nii.gz'))
    
    labels = [label == i for i in range(1,5)]
    SimpleElastix.SetMovingImage(image)
    SimpleElastix.Execute()
    transformation_map = SimpleElastix.GetTransformParameterMap()
    #print(transformation_map[0])
    #transformation_map[0]["ResampleInterpolator"] = ["FinalNearestNeighborInterpolator"]
    
    tr_lbl = [sitk.GetArrayFromImage(sitk.Transformix(l, transformation_map) > 0.5) for l in labels]
    tr_lbl = np.stack([np.full(shape=tr_lbl[0].shape, fill_value=0.5, dtype=np.float32),]+tr_lbl,axis=0)
    tr_lbl = np.argmax(tr_lbl,axis=0)
    tr_lbl = sitk.Cast(sitk.GetImageFromArray(tr_lbl),sitk.sitkUInt8)
    resultLabels.push_back(tr_lbl)
    print(image_name)

fixedLabel = sitk.LabelVoting(resultLabels)

Patient_02
Patient_03
Patient_04
Patient_05
Patient_06
Patient_07
Patient_08
Patient_09
Patient_10
Patient_11
Patient_12
Patient_13
Patient_14
Patient_15
Patient_16
Patient_17
Patient_18
Patient_19
Patient_20
Patient_21
Patient_22
Patient_23
Patient_24
Patient_25
Patient_26
Patient_27
Patient_28
Patient_29
Patient_30
Patient_31
Patient_32
Patient_33
Patient_34
Patient_35
Patient_36
Patient_37
Patient_38
Patient_39
Patient_40


In [8]:
sitk.WriteImage(fixedLabel, 'fixed_output_label.nii.gz', True) 